In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式（不打开浏览器界面）
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速

# 使用 webdriver-manager 自动下载驱动
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 打开网页
url = "https://select.pdgzf.com/houseLists"
driver.get(url)

# 等待页面加载，获取页面源代码
driver.implicitly_wait(10)  # 等待 10 秒，确保网页完全加载

# 获取网页源代码
html = driver.page_source

# 使用 BeautifulSoup 解析 HTML
soup = BeautifulSoup(html, "html.parser")

# 找到所有 class 为 "clearfix screen-item marB10" 的 div 元素
div_elements = soup.find_all('div', {'class': 'clearfix screen-item marB10'})

result_div = None

df = pd.DataFrame()

# 遍历所有 div 元素
for div in div_elements:
    # 在当前 div 中查找 h4 标签
    h4_element = div.find('h4')
    # 如果找到了 h4 标签，并且文本为 "小区"，则将当前 div 存储起来
    if h4_element and h4_element.text == '小区：':
        result_div = div
        break  # 找到目标 div 后，跳出循环

if result_div:
    # 在找到的 div 中查找 ul 列表
    ul_element = result_div.find('ul', {'class': 'clearfix fl'})

    if ul_element:
        # 提取 li 元素
        li_elements = ul_element.find_all('li')

        # 将 li 元素存储在列表中
        data = []
        for li in li_elements:
            data.append(li.text.strip())

        # 创建 Pandas DataFrame
        df = pd.DataFrame(data, columns=['name'])
    else:
        print("未找到 ul 元素。")
else:
    print("未找到包含指定 h4 标签的 div 元素。")


In [19]:
df_houses = df.drop(index=0)
df_houses['address'] = df_houses['name'].str.extract(r'(.+?)(?:[\(（].*?)?$') # Key change
df_houses['address'] = '上海市浦东新区' + df_houses['address']

In [32]:
import requests
import time
import pandas as pd

# 你的高德 API Key
API_KEY = '37ca6fad9ef2f4a0272eb39d82cc24cb'

# 定义函数：调用高德地理编码 API 获取经纬度
def get_geocode(address, api_key, counter):
    url = f'https://restapi.amap.com/v3/geocode/geo?address={address}&key={api_key}'
    response = requests.get(url)
    
    # 每调用 3 次 API，休眠 1 秒
    counter['count'] += 1
    if counter['count'] % 3 == 0:
        time.sleep(1)  # 休眠 1 秒
    
    if response.status_code == 200:
        result = response.json()
        if result['status'] == '1' and len(result['geocodes']) > 0:
            location = result['geocodes'][0]['location']  # 提取经纬度
            return location
    return None

# 初始化计数器
counter = {'count': 0}

# 为 DataFrame 添加经纬度列
df_houses['location'] = df_houses['address'].apply(lambda x: get_geocode(x, API_KEY, counter))

# 拆分 location 列为 latitude 和 longitude
df_houses[['longitude', 'latitude']] = df_houses['location'].str.split(',', expand=True)

# 删除临时的 location 列
df_houses.drop(columns=['location'], inplace=True)

# 打印结果
df_houses

KeyboardInterrupt: 

In [21]:
df_houses.to_csv('houses.csv', index=False, encoding='utf-8-sig')

In [23]:
df_shuttle = pd.read_excel('shuttle.xlsx', sheet_name='Sheet1')

# 初始化计数器
counter = {'count': 0}

# 为 DataFrame 添加经纬度列
df_shuttle['location'] = df_shuttle['name'].apply(lambda x: get_geocode(x, API_KEY, counter))

# 拆分 location 列为 latitude 和 longitude
df_shuttle[['longitude', 'latitude']] = df_shuttle['location'].str.split(',', expand=True)

# 删除临时的 location 列
df_shuttle.drop(columns=['location'], inplace=True)

# 打印结果
df_shuttle

,班车号,路线,车牌,车型,到站时间,name,longitude,latitude
0,1号,中山公园,沪FD0815,金旅,07:00:00,延安西路程家桥路,121.375117,31.185275
1,NaN,NaN,NaN,NaN,07:20:00,长宁路娄山关路公交站,121.404344,31.210724
2,NaN,NaN,NaN,NaN,07:30:00,凯旋路汇川路公交站,121.417364,31.216742
3,2号,梅陇,沪FA1511,大通,06:50:00,莘庄(莘松路莘东路) 公交站,121.376991,31.108371
4,NaN,NaN,NaN,NaN,07:00:00,龙茗路公交站,121.387863,31.134375
...,...,...,...,...,...,...,...,...
115,19号,张庙(翻版),沪EB7900,大通,06:15:00,长江西路虎林路(张庙)公交站,121.450263,31.336744
116,NaN,NaN,NaN,NaN,06:25:00,长江西路淞南路公交站,120.671878,31.185729
117,NaN,NaN,NaN,NaN,06:45:00,宝林路牡丹江路公交站,121.485425,31.400846
118,NaN,NaN,NaN,NaN,07:00:00,逸仙路江湾镇公交站,121.485730,31.302570


In [24]:
df_shuttle.to_csv('shuttle.csv', index=False, encoding='utf-8-sig')

In [33]:
# 将 DataFrame 转换为 JavaScript 数组
def df_to_js_array(df, name):
    js_array = df.to_json(orient='records', force_ascii=False)
    return f"var {name} = {js_array};"

# 生成 JavaScript 数据
houses_js = df_to_js_array(df_houses, 'houses')
shuttle_js = df_to_js_array(df_shuttle, 'shuttle')

# html 文本
html_content = f'''
<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <title>浦东公租房与班车地图</title>
    <style>
        #map-container {{
            width: 100%;
            height: 800px;
        }}
    </style>
    <script type="text/javascript">
        window._AMapSecurityConfig = {{
            securityJsCode: "cf3973bb8270302fb372cbe696992ad5",
        }};
    </script>
    <script src="https://webapi.amap.com/maps?v=2.0&key=9f768a0efd36b710e294263a060c41f0"></script>
    <!-- 引入 AMapUI 的 JS 文件 -->
    <script src="https://webapi.amap.com/ui/1.1/main.js"></script>
    <!-- 引入 jQuery（AMapUI 依赖） -->
    <script src="https://cdn.bootcdn.net/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
</head>

<body>
    <div id="map-container"></div>
    <script>
        // 嵌入 CSV 数据
        {houses_js}
        {shuttle_js}

        // 初始化地图
        var map = new AMap.Map('map-container', {{
            zoom: 12, // 缩放级别
            center: [121.612600, 31.243856] // 地图中心点（你上班的位置）
        }});

        //设置DomLibrary，jQuery或者Zepto
        AMapUI.setDomLibrary($);

        //加载BasicControl，loadUI的路径参数为模块名中 'ui/' 之后的部分
        AMapUI.loadUI(['control/BasicControl'], function (BasicControl) {{
            //缩放控件
            map.addControl(new BasicControl.Zoom({{
                position: 'lt', //left top，左上角
                showZoomNum: true //显示zoom值
            }}));

            //图层切换控件
            map.addControl(new BasicControl.LayerSwitcher({{
                position: 'rt' //right top，右上角
            }}));

            //实时交通控件
            map.addControl(new BasicControl.Traffic({{
                position: 'lb' //left bottom, 左下角
            }}));
        }});

        // 加载 AMapUI 的 SimpleInfoWindow
        AMapUI.loadUI(['overlay/AwesomeMarker', 'overlay/SimpleInfoWindow'], function (AwesomeMarker, SimpleInfoWindow) {{

            // 标注你上班的位置（工作地点图标）
            var workMarker = new AwesomeMarker({{
                awesomeIcon: 'arrows', // 图标类型
                iconLabel: {{
                    style: {{
                        color: '#333', // 图标颜色设置为黑色
                        fontSize: '15px' // 设置字号
                    }}
                }},
                // 自定义背景样式
                iconStyle: 'orange',
                map: map,
                title: '上班地点',
                position: [121.612600, 31.243856]
            }});

            // 为上班地点添加点击事件
            workMarker.on('click', function() {{
                var infoWindow = new SimpleInfoWindow({{
                    infoTitle: '公司',
                    infoBody: '申美饮料。',
                    offset: [0, -30] // 信息窗口的偏移量
                }});
                infoWindow.open(map, workMarker.getPosition());
            }});

            // 标注公租房位置（房子图标）
            houses.forEach(function (house) {{
                var houseMarker = new AwesomeMarker({{
                    awesomeIcon: 'home', // 图标类型
                    iconLabel: {{
                        style: {{
                            color: '#333', // 图标颜色设置为黑色
                            fontSize: '16px' // 设置字号
                        }}
                    }},
                    // 自定义背景样式
                    iconStyle: {{
                        src: 'https://webapi.amap.com/theme/v1.3/markers/b/mark_b.png',
                        style: {{
                            width: '22px',
                            height: '32px'
                        }}
                    }},
                    map: map,
                    title: house.name,
                    position: [house.longitude, house.latitude]
                }});

                // 为公租房添加点击事件
                houseMarker.on('click', function() {{
                    var infoWindow = new SimpleInfoWindow({{
                        infoTitle: house.name,
                        infoBody: house.address,
                        offset: [0, -30] // 信息窗口的偏移量
                    }});
                    infoWindow.open(map, houseMarker.getPosition());
                }});
            }});
            
            // 标注班车上下车点（公交车图标）
            shuttle.forEach(function (point) {{
                var shuttleMarker = new AwesomeMarker({{
                    awesomeIcon: 'bus', // 图标类型
                    iconLabel: {{
                        style: {{
                            color: '#333', // 图标颜色设置为黑色
                            fontSize: '16px' // 设置字号
                        }}
                    }},
                    // 自定义背景样式
                    iconStyle: {{
                        src: 'https://webapi.amap.com/theme/v1.3/markers/b/mark_y.png',
                        style: {{
                            width: '22px',
                            height: '32px'
                        }}
                    }},
                    map: map,
                    title: point.name + point.到站时间,
                    position: [point.longitude, point.latitude]
                }});

                // 为班车点添加点击事件
                shuttleMarker.on('click', function() {{
                    var infoWindow = new SimpleInfoWindow({{
                        infoTitle: point.name,
                        infoBody: `班车号：${{point.班车号}}<br>到站时间：${{point.到站时间}}`,
                        offset: [0, -30] // 信息窗口的偏移量
                    }});
                    infoWindow.open(map, shuttleMarker.getPosition());
                }});
            }});

        }});
    </script>
</body>

</html>
'''

# 保存 HTML 文件
with open('index.html', 'w', encoding='utf-8') as f:
    f.write(html_content)

print("地图已生成，请打开 index.html 查看。")

地图已生成，请打开 index.html 查看。
